In [ ]:
import cupy as cp
import numpy as np
from time import perf_counter
import time
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
from pyqcu import eigen
from pyqcu.set import  params,argv,set_ptrs
import cupyx.scipy.sparse.linalg as csla
import scipy.linalg as sla

print('My rank is ', define.rank)

In [ ]:
gauge_filename = f"quda_wilson-dslash-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.bin"
print("Gauge filename:", gauge_filename)
gauge = cp.fromfile(gauge_filename, dtype=cp.complex64,
                    count=params[define._LAT_XYZT_]*define._LAT_DCC_)
gauge = io.gauge2ccdptzyx(gauge, params)
print("Gauge:", gauge)
print("Gauge data:", gauge.data)
print("Gauge shape:", gauge.shape)

In [ ]:
qcu.applyInitQcu(set_ptrs, params, argv)

In [ ]:
gauge.size

In [ ]:
# n = params[define._LAT_XYZT_] * define._LAT_HALF_SC_


# def matvec(src):
#     src = src.astype(cp.complex64)
#     dest = cp.zeros_like(src)
#     qcu.applyWilsonCgDslashQcu(
#         dest, src, gauge, set_ptrs, params)
#     src = src.astype(cp.complex128)
#     dest = dest.astype(cp.complex128)
#     return dest


# eigen_solver = eigen.solver(n=n, k=3, matvec=matvec, dtype=cp.complex128)

# eigenvalues, eigenvectors = eigen_solver.run()

In [6]:
n = params[define._LAT_XYZT_] * define._LAT_HALF_SC_


def matvec(src):
    dest = cp.zeros_like(src)
    qcu.applyWilsonCgDslashQcu(
        dest, src, gauge, set_ptrs, params)
    return dest


eigen_solver = eigen.solver(n=n, k=3, matvec=matvec, dtype=gauge.dtype)

eigenvalues, eigenvectors = eigen_solver.run()

eigen_index: 0, iter: 0, alpha: 0.000000000, beta: 1.000000000, tol: inf, lambda: 1.529528260, degree: 20
eigen_index: 0, iter: 1, alpha: 0.764764130, beta: 1.000000000, tol: 8.663684e+01, lambda: 0.017453028, degree: 30
eigen_index: 0, iter: 2, alpha: 0.764764130, beta: 1.000000000, tol: 5.660984e-01, lambda: 0.011144273, degree: 30
eigen_index: 0, iter: 3, alpha: 0.764764130, beta: 1.000000000, tol: 5.350064e-01, lambda: 0.007260082, degree: 30
eigen_index: 0, iter: 4, alpha: 0.764764130, beta: 1.000000000, tol: 5.613898e-01, lambda: 0.004649756, degree: 30
eigen_index: 0, iter: 5, alpha: 0.764764130, beta: 1.000000000, tol: 5.610066e-01, lambda: 0.002978691, degree: 30
eigen_index: 0, iter: 6, alpha: 0.764764130, beta: 1.000000000, tol: 8.219077e-01, lambda: 0.001634930, degree: 45
eigen_index: 0, iter: 7, alpha: 0.764764130, beta: 1.000000000, tol: 5.538845e-01, lambda: 0.001052157, degree: 45
eigen_index: 0, iter: 8, alpha: 0.764764130, beta: 1.000000000, tol: 2.982412e-01, lambda

KeyboardInterrupt: 

In [ ]:
eigenvalues

In [ ]:
eigenvectors

In [ ]:
eigen_solver.matvec(eigenvectors[0])

In [ ]:
eigenvectors[0]*eigenvalues[0]

In [ ]:
# Verify results
print("Computed eigenvalues:")
for i, ev in enumerate(eigenvalues):
    print(f"λ_{i} = {ev:.8f}")
    # Verify eigenvector
    v = eigenvectors[i]
    w = cp.zeros_like(v)
    w = eigen_solver.matvec(v)
    error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
    print(f"Relative error: {error:.2e}")
    j = i+1
    if j == len(eigenvalues):
        j = 0
    print(
        f"Diff between λ_{i} and λ_{j}: {cp.linalg.norm(eigenvectors[i] - eigenvectors[j])/cp.linalg.norm(eigenvectors[i]):.2e}")

In [19]:
eigenvalues.tofile("eigenvalues-dev33.bin")
eigenvectors.tofile("eigenvectors-dev33.bin")

In [12]:
# qcu.applyEndQcu(set_ptrs, params)